In [18]:
%pip install --upgrade --quiet langchain-openai



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [19]:
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
import numpy as np
import json
import tiktoken


In [20]:

# Get the current working directory
current_dir = os.getcwd()

# Go up two levels to reach the directory containing keys.env
parent_dir = os.path.dirname(os.path.dirname(current_dir))

# Construct the full path to keys.env
env_path = os.path.join(parent_dir, 'keys.env')

print(f"Looking for keys.env at: {env_path}")

# Check if the file exists
if os.path.exists(env_path):
    print("keys.env file found.")
    # Load environment variables from `keys.env`
    load_dotenv(env_path)

    # Accessing the variables
    bias1_key1 = os.getenv('BIAS1_KEY1')
    azure_openai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
    azure_openai_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
    azure_openai_api_version = os.getenv('AZURE_OPENAI_API_VERSION')
    azure_openai_chat_deployment_name_gpt3 = os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME_GPT3')
    azure_openai_embed_deployment_large = os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME_EMBED_LARGE')
    huggingfacehub_api_token = os.getenv('HUGGINGFACEHUB_API_TOKEN')



Looking for keys.env at: /Users/kenzabenkirane/Desktop/GitHub/24ucl_thesis/bias_llm_clinical_challenge/keys.env
keys.env file found.


In [21]:
%pip install --upgrade --quiet langchain-openai



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [22]:
# --- with LangChain
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    api_key=azure_openai_api_key,
    azure_deployment=azure_openai_embed_deployment_large,
    openai_api_version=azure_openai_api_version
)


In [23]:
# =========== Helper functions ===========
def get_embedding(text):
    """
    This function retrieves the embedding for a given text using the specified model.
    """
    return embeddings.embed_query(text)

def cosine_similarity(a, b):
    """
    This function calculates the cosine similarity between two embedding vectors.
    """
    if a is None or b is None:
        return None
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def calculate_similarity(text1, text2):
    embedding1 = get_embedding(text1)
    embedding2 = get_embedding(text2)
    
    if embedding1 is None or embedding2 is None:
        print("Error: Unable to generate embeddings for one or both texts.")
        return None
    
    return cosine_similarity(embedding1, embedding2)

def num_tokens_from_string(string: str, encoding_name: str = "cl100k_base") -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def truncate_text(text: str, max_tokens: int = 8192, encoding_name: str = "cl100k_base") -> str:
    """Truncates the text to the specified maximum number of tokens."""
    encoding = tiktoken.get_encoding(encoding_name)
    tokens = encoding.encode(text)
    if len(tokens) > max_tokens:
        truncated_tokens = tokens[:max_tokens]
        truncated_text = encoding.decode(truncated_tokens)
        warnings.warn(f"Text truncated from {len(tokens)} to {max_tokens} tokens.")
        return truncated_text
    return text


In [24]:
# Example usage
text1 = "this is an awesome test document"
text2 = "this is an aweful test document"
query_result = calculate_similarity(text1, text2)

if query_result is not None:
    print(f"Cosine similarity: {query_result}")
else:
    print("Failed to calculate similarity.")
    

Cosine similarity: 0.8479392455403287


In [25]:
embed_test=get_embedding("Testing")


In [26]:
type(embed_test)


list

# Experiment 1

## 1a Gender original

In [111]:
experiment_name='exp1_G'
experiment_folder = os.path.join(parent_dir, 'results/experiment1/gender/cleaned_results') #!!!
starts_with = 'llm_' 


In [97]:
df_path=os.path.join(experiment_folder,'all_othercolumns.csv')
df=pd.read_csv(df_path)
df.head()


,test_image,link,opb_shuffled,key_id,opc_shuffled,clinical_question,pregnancy,age_group,answer_idx_shuffled,woman_health,...,opc,opb,diagnosis,experiment,opa_shuffled,explanation,augmented_gender,original_gender,opd_shuffled,answer_idx
0,1,https://jamanetwork.com/journals/jamaoncology/...,Kimura disease,4_G_o_o_m,Classic Hodgkin lymphoma,what is your diagnosis?,0,31-40,d,0,...,T-cell acute lymphoblastic lymphoma/leukemia,Classic Hodgkin lymphoma,D. Myeloid/lymphoid neoplasms with eosinophili...,G,T-cell acute lymphoblastic lymphoma/leukemia,The differential diagnoses in young men with e...,original,male,Myeloid/lymphoid neoplasms with eosinophilia a...,D
1,1,https://jamanetwork.com/journals/jamaoncology/...,Kimura disease,4_G_a_f_m,Classic Hodgkin lymphoma,what is your diagnosis?,0,31-40,d,0,...,T-cell acute lymphoblastic lymphoma/leukemia,Classic Hodgkin lymphoma,D. Myeloid/lymphoid neoplasms with eosinophili...,G,T-cell acute lymphoblastic lymphoma/leukemia,The differential diagnoses in young men with e...,female,male,Myeloid/lymphoid neoplasms with eosinophilia a...,D
2,1,https://jamanetwork.com/journals/jamaoncology/...,Kimura disease,4_G_a_n_m,Classic Hodgkin lymphoma,what is your diagnosis?,0,31-40,d,0,...,T-cell acute lymphoblastic lymphoma/leukemia,Classic Hodgkin lymphoma,D. Myeloid/lymphoid neoplasms with eosinophili...,G,T-cell acute lymphoblastic lymphoma/leukemia,The differential diagnoses in young men with e...,neutral,male,Myeloid/lymphoid neoplasms with eosinophilia a...,D
3,0,https://jamanetwork.com/journals/jamaoncology/...,Medication adverse effect,11_G_o_o_f,Melanoma,what is your diagnosis?,0,51-60,b,0,...,Medication adverse effect,Melanoma,C. Medication adverse effect,G,Laugier-Hunziker syndrome,Oral hyperpigmentation is a rare adverse effec...,original,female,Oral involvement of mycosis fungoides,C
4,0,https://jamanetwork.com/journals/jamaoncology/...,Medication adverse effect,11_G_a_m_f,Melanoma,what is your diagnosis?,0,51-60,b,0,...,Medication adverse effect,Melanoma,C. Medication adverse effect,G,Laugier-Hunziker syndrome,Oral hyperpigmentation is a rare adverse effec...,male,female,Oral involvement of mycosis fungoides,C


In [98]:
print(f"Columns: {df.columns}")
print(f"Shape: {df.shape}")


Columns: Index(['test_image', 'link', 'opb_shuffled', 'key_id', 'opc_shuffled',
       'clinical_question', 'pregnancy', 'age_group', 'answer_idx_shuffled',
       'woman_health', 'gender', 'version_gender', 'version_binary', 'case_id',
       'field', 'figure', 'opd', 'test_lab', 'test_other', 'ethnicity',
       'version', 'age', 'normalized_question', 'case', 'opa', 'gender_int',
       'question', 'answer', 'opc', 'opb', 'diagnosis', 'experiment',
       'opa_shuffled', 'explanation', 'augmented_gender', 'original_gender',
       'opd_shuffled', 'answer_idx'],
      dtype='object')
Shape: (600, 38)


In [99]:
df['case_id'].nunique()


200

#### 1a.a. Explanation -> per unique case_id

In [104]:
import pandas as pd
import numpy as np
import os
import json

def process_dataframe_per_caseid(df, get_embedding, cosine_similarity, max_tokens=8192, experiment_name='exp1'):
    # Create df_uniquecaseid with the first row for each unique case_id
    df_uniquecaseid = df.groupby('case_id').first().reset_index()
    
    # Initialize new columns
    new_columns = ['explanation_truncated', 'explanation_tokens', 'explanation_embedding']
    response_columns = [col for col in df_uniquecaseid.columns if 'response' in col]
    for col in response_columns:
        new_columns.extend([f'{col}_truncated', f'{col}_tokens', f'{col}_embedding', f'{col}_embedding_sim'])
    
    for col in new_columns:
        if col not in df_uniquecaseid.columns:
            df_uniquecaseid[col] = None
    
    # Process df_uniquecaseid
    total_rows = len(df_uniquecaseid)
    for i, (index, row) in enumerate(df_uniquecaseid.iterrows(), 1):
        # Create truncated versions if necessary
        df_uniquecaseid.at[index, 'explanation_truncated'] = truncate_text(row['explanation'], max_tokens)
        df_uniquecaseid.at[index, 'explanation_tokens'] = num_tokens_from_string(row['explanation'])
        
        # Calculate embedding for explanation
        explanation_embedding = get_embedding(df_uniquecaseid.at[index, 'explanation_truncated'])
        df_uniquecaseid.at[index, 'explanation_embedding'] = json.dumps(explanation_embedding)
        
        # Process response columns
        for col in response_columns:
            df_uniquecaseid.at[index, f'{col}_truncated'] = truncate_text(row[col], max_tokens)
            df_uniquecaseid.at[index, f'{col}_tokens'] = num_tokens_from_string(row[col])
            response_embedding = get_embedding(df_uniquecaseid.at[index, f'{col}_truncated'])
            df_uniquecaseid.at[index, f'{col}_embedding'] = json.dumps(response_embedding)
            df_uniquecaseid.at[index, f'{col}_embedding_sim'] = cosine_similarity(
                explanation_embedding,
                response_embedding
            )
        
        # Print progress and save to CSV every 10%
        if i % (total_rows // 10) == 0 or i == total_rows:
            print(f"Processed {i}/{total_rows} rows ({i/total_rows*100:.2f}%)")
            df_uniquecaseid.to_csv(f'progress/df_{experiment_name}_uniquecaseid_progress_row{i}.csv', index=False)

    # Process the original dataframe
    df_embed_uniquecaseid = df.copy()
    
    # Initialize new columns in df_embed_uniquecaseid
    for col in new_columns:
        if col not in df_embed_uniquecaseid.columns:
            df_embed_uniquecaseid[col] = None
    
    # Map new columns from df_uniquecaseid to df_embed_uniquecaseid
    columns_to_map = ['explanation_embedding'] + new_columns
    for col in columns_to_map:
        df_embed_uniquecaseid[col] = df_embed_uniquecaseid['case_id'].map(dict(zip(df_uniquecaseid['case_id'], df_uniquecaseid[col])))
    
    # Save the final dataframe
    df_embed_uniquecaseid=df_embed_uniquecaseid[['case_id','explanation','explanation_truncated','explanation_tokens','explanation_embedding']]

    df_embed_uniquecaseid.to_csv(f'{experiment_folder}/all_othercolumns_embed.csv', index=False)

    
    return df_embed_uniquecaseid



In [105]:
df_embed_uniquecaseid = process_dataframe_per_caseid(df, get_embedding, cosine_similarity, experiment_name=experiment_name)


Processed 20/200 rows (10.00%)
Processed 40/200 rows (20.00%)
Processed 60/200 rows (30.00%)
Processed 80/200 rows (40.00%)
Processed 100/200 rows (50.00%)
Processed 120/200 rows (60.00%)
Processed 140/200 rows (70.00%)
Processed 160/200 rows (80.00%)
Processed 180/200 rows (90.00%)
Processed 200/200 rows (100.00%)


In [74]:
df_embed_uniquecaseid


,case_id,explanation,explanation_truncated,explanation_tokens,explanation_embedding
0,4,The differential diagnoses in young men with e...,The differential diagnoses in young men with e...,959,"[-0.01970686763525009, 0.018883252516388893, -..."
1,4,The differential diagnoses in young men with e...,The differential diagnoses in young men with e...,959,"[-0.01970686763525009, 0.018883252516388893, -..."
2,4,The differential diagnoses in young men with e...,The differential diagnoses in young men with e...,959,"[-0.01970686763525009, 0.018883252516388893, -..."
3,11,Oral hyperpigmentation is a rare adverse effec...,Oral hyperpigmentation is a rare adverse effec...,871,"[-0.03791378438472748, 0.002430149121209979, -..."
4,11,Oral hyperpigmentation is a rare adverse effec...,Oral hyperpigmentation is a rare adverse effec...,871,"[-0.03791378438472748, 0.002430149121209979, -..."
...,...,...,...,...,...
595,1521,In this case of group C streptococci–related l...,In this case of group C streptococci–related l...,990,"[-0.005955410655587912, 0.0438690185546875, 0...."
596,1521,In this case of group C streptococci–related l...,In this case of group C streptococci–related l...,990,"[-0.005955410655587912, 0.0438690185546875, 0...."
597,1522,The multiple incidences of osteolytic lesions ...,The multiple incidences of osteolytic lesions ...,646,"[0.017044944688677788, 0.014894481748342514, -..."
598,1522,The multiple incidences of osteolytic lesions ...,The multiple incidences of osteolytic lesions ...,646,"[0.017044944688677788, 0.014894481748342514, -..."


In [107]:
# === Analysis

# --- 2/ Check the NA values
print(f"NaN values in the dataframe: {df_embed_uniquecaseid.isna().sum().sum()}")

# ---3/ Check the number of unique case_id
print(f"Number of unique case_id: {df_embed_uniquecaseid['case_id'].nunique()}")

# ---4/ Check the number of columns
print(f"Columns: {df_embed_uniquecaseid.columns}")


NaN values in the dataframe: 0
Number of unique case_id: 200
Columns: Index(['case_id', 'explanation', 'explanation_truncated', 'explanation_tokens',
       'explanation_embedding'],
      dtype='object')


### 1a.b. LLM output --> Per llm

In [ ]:
import pandas as pd
import numpy as np
import os
import json
from tqdm import tqdm

def process_llm_csvs(get_embedding, cosine_similarity, truncate_text, num_tokens_from_string, max_tokens=8192):
    print("1/ Loading the data")
    dfs = []
    file_paths = []
    for file in os.listdir(experiment_folder):
        if file.startswith('llm_') and file.endswith('.csv'):
            file_path = os.path.join(experiment_folder, file)
            df = pd.read_csv(file_path)
            dfs.append(df)
            file_paths.append(file_path)
    
    print("2/ Processing DataFrames")
    for df, file_path in tqdm(zip(dfs, file_paths), total=len(dfs), desc="Processing files"):
        print(f"\nProcessing: {os.path.basename(file_path)}")
        
        # 2a/ Get the embedding of explanation from df_embed_uniquecaseid
        print("2a/ Adding explanation embeddings")
        # Create a dictionary mapping case_id to embedding, keeping only the first occurrence of each case_id
        embedding_dict = df_embed_uniquecaseid.groupby('case_id')['explanation_embedding'].first().to_dict()
        df['explanation_embedding'] = df['case_id'].map(embedding_dict)
        
        # 2b/ Calculate the embedding of each response and the cosine similarity with the explanation
        print("2b/ Calculating embeddings and similarities for responses")
        columns_to_evaluate = [col for col in df.columns if 'response' in col]
        for col in tqdm(columns_to_evaluate, desc="Processing columns"):
            df[f'{col}_truncated'] = df[col].apply(lambda x: truncate_text(x, max_tokens))
            df[f'{col}_tokens'] = df[col].apply(num_tokens_from_string)
            df[f'{col}_embedding'] = df[f'{col}_truncated'].apply(get_embedding).apply(json.dumps)
            df[f'{col}_embedding_sim'] = df.apply(lambda x: cosine_similarity(
                json.loads(x['explanation_embedding']), 
                json.loads(x[f'{col}_embedding'])
            ), axis=1)
        
        # 2c/ Save the DataFrame
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        output_path = os.path.join(os.path.dirname(file_path), f"{base_name}_embed.csv")
        df.to_csv(output_path, index=False)
        print(f"Saved processed data to: {output_path}")

    print("3/ Done processing all DataFrames")

# Usage
process_llm_csvs(get_embedding, cosine_similarity, truncate_text, num_tokens_from_string)


In [113]:
# import pandas as pd
# import numpy as np
# import os
# import json
# from tqdm import tqdm

# def process_llm_csvs(get_embedding, cosine_similarity, truncate_text, num_tokens_from_string, max_tokens=8192):
#     print("1/ Loading the data")
#     dfs = []
#     file_paths = []
#     for file in os.listdir(experiment_folder):
#         if file.startswith('llm_') and file.endswith('.csv'):
#             file_path = os.path.join(experiment_folder, file)
#             df = pd.read_csv(file_path)
#             dfs.append(df)
#             file_paths.append(file_path)
    
#     print("2/ Processing DataFrames")
#     for df, file_path in tqdm(zip(dfs, file_paths), total=len(dfs), desc="Processing files"):
#         print(f"\nProcessing: {os.path.basename(file_path)}")
        
#         # 2a/ Get the embedding of explanation from df_embed_uniquecaseid
#         print("2a/ Adding explanation embeddings")
#         # Create a dictionary mapping case_id to embedding, keeping only the first occurrence of each case_id
#         embedding_dict = df_embed_uniquecaseid.groupby('case_id')['explanation_embedding'].first().to_dict()
#         df['explanation_embedding'] = df['case_id'].map(embedding_dict)
        
#         # 2b/ Calculate the embedding of each response and the cosine similarity with the explanation
#         print("2b/ Calculating embeddings and similarities for responses")
#         columns_to_evaluate = [col for col in df.columns if 'response' in col]
#         for col in tqdm(columns_to_evaluate, desc="Processing columns"):
#             df[f'{col}_truncated'] = df[col].apply(lambda x: truncate_text(x, max_tokens))
#             df[f'{col}_tokens'] = df[col].apply(num_tokens_from_string)
#             df[f'{col}_embedding'] = df[f'{col}_truncated'].apply(get_embedding).apply(json.dumps)
#             df[f'{col}_embedding_sim'] = df.apply(lambda x: cosine_similarity(
#                 json.loads(x['explanation_embedding']), 
#                 json.loads(x[f'{col}_embedding'])
#             ), axis=1)
        
#         # 2c/ Save the DataFrame
#         output_path = file_path + "_embed.csv"
#         df.to_csv(output_path, index=False)
#         print(f"Saved processed data to: {output_path}")

#     print("3/ Done processing all DataFrames")

# # Usage
# process_llm_csvs(get_embedding, cosine_similarity, truncate_text, num_tokens_from_string)


1/ Loading the data
2/ Processing DataFrames


Processing files:   0%|          | 0/4 [00:00<?, ?it/s]


Processing: llm_gpt4o.csv
2a/ Adding explanation embeddings
2b/ Calculating embeddings and similarities for responses


Processing files:  25%|██▌       | 1/4 [03:19<09:58, 199.43s/it]

Saved processed data to: /Users/kenzabenkirane/Desktop/GitHub/24ucl_thesis/bias_llm_clinical_challenge/results/experiment1/gender/cleaned_results/llm_gpt4o.csv_embed.csv

Processing: llm_gpt4omini.csv
2a/ Adding explanation embeddings
2b/ Calculating embeddings and similarities for responses


...

Saved processed data to: /Users/kenzabenkirane/Desktop/GitHub/24ucl_thesis/bias_llm_clinical_challenge/results/experiment1/gender/cleaned_results/llm_gpt4omini.csv_embed.csv

Processing: llm_gpt3.csv
2a/ Adding explanation embeddings
2b/ Calculating embeddings and similarities for responses


Processing files:  75%|███████▌  | 3/4 [10:07<03:22, 202.47s/it]

Saved processed data to: /Users/kenzabenkirane/Desktop/GitHub/24ucl_thesis/bias_llm_clinical_challenge/results/experiment1/gender/cleaned_results/llm_gpt3.csv_embed.csv

Processing: llm_gpt4turbo.csv
2a/ Adding explanation embeddings
2b/ Calculating embeddings and similarities for responses


Processing files: 100%|██████████| 4/4 [13:27<00:00, 201.97s/it]

Saved processed data to: /Users/kenzabenkirane/Desktop/GitHub/24ucl_thesis/bias_llm_clinical_challenge/results/experiment1/gender/cleaned_results/llm_gpt4turbo.csv_embed.csv
3/ Done processing all DataFrames


## 1b. GxE

In [116]:
experiment_name='exp1_GxE'
experiment_folder = os.path.join(parent_dir, 'results/experiment1/genderxethnicity/cleaned_results') #!!!
starts_with = 'llm_' 


In [117]:
df_path=os.path.join(experiment_folder,'all_othercolumns.csv')
df=pd.read_csv(df_path)
df.head()


,clinical_question,test_lab,question,gender_int,opb,answer_idx_shuffled,explanation,opa_shuffled,version_gender,version_binary,...,normalized_question,age_group,link,test_image,woman_health,figure,opc,key_id,case,age
0,what is your diagnosis?,1,A 46-year-old man presented with a left should...,1,Basal cell carcinoma,b,The biomarker SOX10 is highly sensitive and sp...,Squamous cell carcinoma,male_augmented,augmented,...,What is your diagnosis?,41-50,https://jamanetwork.com/journals/jamaoncology/...,1,0,1,Melanoma,28_GxE_a_W_m,A 46-year-old White man presented with a left ...,46.0
1,what is your diagnosis?,1,A 46-year-old man presented with a left should...,-1,Basal cell carcinoma,b,The biomarker SOX10 is highly sensitive and sp...,Squamous cell carcinoma,female_augmented,augmented,...,What is your diagnosis?,41-50,https://jamanetwork.com/journals/jamaoncology/...,1,0,1,Melanoma,28_GxE_a_W_m,A 46-year-old White woman presented with a lef...,46.0
2,what is your diagnosis?,1,A 46-year-old man presented with a left should...,0,Basal cell carcinoma,b,The biomarker SOX10 is highly sensitive and sp...,Squamous cell carcinoma,neutral_augmented,augmented,...,What is your diagnosis?,41-50,https://jamanetwork.com/journals/jamaoncology/...,1,0,1,Melanoma,28_GxE_a_W_m,A 46-year-old White patient presented with a l...,46.0
3,what is your diagnosis?,1,A 46-year-old man presented with a left should...,1,Basal cell carcinoma,b,The biomarker SOX10 is highly sensitive and sp...,Squamous cell carcinoma,male_augmented,augmented,...,What is your diagnosis?,41-50,https://jamanetwork.com/journals/jamaoncology/...,1,0,1,Melanoma,28_GxE_a_B_m,A 46-year-old Black man presented with a left ...,46.0
4,what is your diagnosis?,1,A 46-year-old man presented with a left should...,-1,Basal cell carcinoma,b,The biomarker SOX10 is highly sensitive and sp...,Squamous cell carcinoma,female_augmented,augmented,...,What is your diagnosis?,41-50,https://jamanetwork.com/journals/jamaoncology/...,1,0,1,Melanoma,28_GxE_a_B_m,A 46-year-old Black woman presented with a lef...,46.0


In [118]:
print(f"Columns: {df.columns}")
print(f"Shape: {df.shape}")


Columns: Index(['clinical_question', 'test_lab', 'question', 'gender_int', 'opb',
       'answer_idx_shuffled', 'explanation', 'opa_shuffled', 'version_gender',
       'version_binary', 'diagnosis', 'pregnancy', 'answer_idx', 'case_id',
       'test_other', 'opc_shuffled', 'opa', 'gender', 'version', 'opd',
       'experiment', 'ethnicity', 'answer', 'augmented_gender', 'field',
       'opb_shuffled', 'opd_shuffled', 'original_gender',
       'version_gender_ethnicity', 'normalized_question', 'age_group', 'link',
       'test_image', 'woman_health', 'figure', 'opc', 'key_id', 'case', 'age'],
      dtype='object')
Shape: (720, 39)


In [119]:
df['case_id'].nunique()


72

#### 1a.a. Explanation -> per unique case_id

In [120]:
import pandas as pd
import numpy as np
import os
import json

def process_dataframe_per_caseid(df, get_embedding, cosine_similarity, max_tokens=8192, experiment_name='exp1'):
    # Create df_uniquecaseid with the first row for each unique case_id
    df_uniquecaseid = df.groupby('case_id').first().reset_index()
    
    # Initialize new columns
    new_columns = ['explanation_truncated', 'explanation_tokens', 'explanation_embedding']
    response_columns = [col for col in df_uniquecaseid.columns if 'response' in col]
    for col in response_columns:
        new_columns.extend([f'{col}_truncated', f'{col}_tokens', f'{col}_embedding', f'{col}_embedding_sim'])
    
    for col in new_columns:
        if col not in df_uniquecaseid.columns:
            df_uniquecaseid[col] = None
    
    # Process df_uniquecaseid
    total_rows = len(df_uniquecaseid)
    for i, (index, row) in enumerate(df_uniquecaseid.iterrows(), 1):
        # Create truncated versions if necessary
        df_uniquecaseid.at[index, 'explanation_truncated'] = truncate_text(row['explanation'], max_tokens)
        df_uniquecaseid.at[index, 'explanation_tokens'] = num_tokens_from_string(row['explanation'])
        
        # Calculate embedding for explanation
        explanation_embedding = get_embedding(df_uniquecaseid.at[index, 'explanation_truncated'])
        df_uniquecaseid.at[index, 'explanation_embedding'] = json.dumps(explanation_embedding)
        
        # Process response columns
        for col in response_columns:
            df_uniquecaseid.at[index, f'{col}_truncated'] = truncate_text(row[col], max_tokens)
            df_uniquecaseid.at[index, f'{col}_tokens'] = num_tokens_from_string(row[col])
            response_embedding = get_embedding(df_uniquecaseid.at[index, f'{col}_truncated'])
            df_uniquecaseid.at[index, f'{col}_embedding'] = json.dumps(response_embedding)
            df_uniquecaseid.at[index, f'{col}_embedding_sim'] = cosine_similarity(
                explanation_embedding,
                response_embedding
            )
        
        # Print progress and save to CSV every 10%
        if i % (total_rows // 10) == 0 or i == total_rows:
            print(f"Processed {i}/{total_rows} rows ({i/total_rows*100:.2f}%)")
            df_uniquecaseid.to_csv(f'progress/df_{experiment_name}_uniquecaseid_progress_row{i}.csv', index=False)

    # Process the original dataframe
    df_embed_uniquecaseid = df.copy()
    
    # Initialize new columns in df_embed_uniquecaseid
    for col in new_columns:
        if col not in df_embed_uniquecaseid.columns:
            df_embed_uniquecaseid[col] = None
    
    # Map new columns from df_uniquecaseid to df_embed_uniquecaseid
    columns_to_map = ['explanation_embedding'] + new_columns
    for col in columns_to_map:
        df_embed_uniquecaseid[col] = df_embed_uniquecaseid['case_id'].map(dict(zip(df_uniquecaseid['case_id'], df_uniquecaseid[col])))
    
    # Save the final dataframe
    df_embed_uniquecaseid=df_embed_uniquecaseid[['case_id','explanation','explanation_truncated','explanation_tokens','explanation_embedding']]

    df_embed_uniquecaseid.to_csv(f'{experiment_folder}/all_othercolumns_embed.csv', index=False)

    
    return df_embed_uniquecaseid



In [121]:
df_embed_uniquecaseid = process_dataframe_per_caseid(df, get_embedding, cosine_similarity, experiment_name=experiment_name)


Processed 7/72 rows (9.72%)
Processed 14/72 rows (19.44%)
Processed 21/72 rows (29.17%)
Processed 28/72 rows (38.89%)
Processed 35/72 rows (48.61%)
Processed 42/72 rows (58.33%)
Processed 49/72 rows (68.06%)
Processed 56/72 rows (77.78%)
Processed 63/72 rows (87.50%)
Processed 70/72 rows (97.22%)
Processed 72/72 rows (100.00%)


In [122]:
df_embed_uniquecaseid


,case_id,explanation,explanation_truncated,explanation_tokens,explanation_embedding
0,28,The biomarker SOX10 is highly sensitive and sp...,The biomarker SOX10 is highly sensitive and sp...,889,"[-0.016631269827485085, 0.0067895445972681046,..."
1,28,The biomarker SOX10 is highly sensitive and sp...,The biomarker SOX10 is highly sensitive and sp...,889,"[-0.016631269827485085, 0.0067895445972681046,..."
2,28,The biomarker SOX10 is highly sensitive and sp...,The biomarker SOX10 is highly sensitive and sp...,889,"[-0.016631269827485085, 0.0067895445972681046,..."
3,28,The biomarker SOX10 is highly sensitive and sp...,The biomarker SOX10 is highly sensitive and sp...,889,"[-0.016631269827485085, 0.0067895445972681046,..."
4,28,The biomarker SOX10 is highly sensitive and sp...,The biomarker SOX10 is highly sensitive and sp...,889,"[-0.016631269827485085, 0.0067895445972681046,..."
...,...,...,...,...,...
715,1184,"Hypofractionated radiotherapy (ie, stereotacti...","Hypofractionated radiotherapy (ie, stereotacti...",593,"[0.005163595546036959, 0.0015109177911654115, ..."
716,1216,Repeated biopsy via thoracotomy revealed small...,Repeated biopsy via thoracotomy revealed small...,1112,"[-0.006852500140666962, 0.023650746792554855, ..."
717,1233,"Brain MRI revealed an enlarged, heterogeneousl...","Brain MRI revealed an enlarged, heterogeneousl...",947,"[-0.03437063843011856, 0.0009665822726674378, ..."
718,1281,The repeated CT scan demonstrated a persistent...,The repeated CT scan demonstrated a persistent...,764,"[-0.03137677535414696, 0.02409636229276657, -0..."


In [123]:
# === Analysis

# --- 2/ Check the NA values
print(f"NaN values in the dataframe: {df_embed_uniquecaseid.isna().sum().sum()}")

# ---3/ Check the number of unique case_id
print(f"Number of unique case_id: {df_embed_uniquecaseid['case_id'].nunique()}")

# ---4/ Check the number of columns
print(f"Columns: {df_embed_uniquecaseid.columns}")


NaN values in the dataframe: 0
Number of unique case_id: 72
Columns: Index(['case_id', 'explanation', 'explanation_truncated', 'explanation_tokens',
       'explanation_embedding'],
      dtype='object')


### 1a.b. LLM output --> Per llm

In [128]:
import pandas as pd
import numpy as np
import os
import json
from tqdm import tqdm

def process_llm_csvs_GxE(get_embedding, cosine_similarity, truncate_text, num_tokens_from_string, max_tokens=8192):
    print("1/ Loading and preprocessing the data")
    dfs = []
    file_paths = []
    for file in os.listdir(experiment_folder):
        if file.startswith('llm_') and file.endswith('.csv'):
            file_path = os.path.join(experiment_folder, file)
            df = pd.read_csv(file_path)
            
            # Preprocess: Convert all 'response' columns to string type and handle NaN values
            response_columns = [col for col in df.columns if 'response' in col]
            for col in response_columns:
                df[col] = df[col].astype(str)
            df = df.fillna('')
            
            dfs.append(df)
            file_paths.append(file_path)
            print(f"Loaded and preprocessed: {file}")
    
    print("2/ Processing DataFrames")
    for df, file_path in tqdm(zip(dfs, file_paths), total=len(dfs), desc="Processing files"):
        print(f"\nProcessing: {os.path.basename(file_path)}")
        
        # 2a/ Get the embedding of explanation from df_embed_uniquecaseid
        print("2a/ Adding explanation embeddings")
        # Create a dictionary mapping case_id to embedding, keeping only the first occurrence of each case_id
        embedding_dict = df_embed_uniquecaseid.groupby('case_id')['explanation_embedding'].first().to_dict()
        df['explanation_embedding'] = df['case_id'].map(embedding_dict)
        
        # 2b/ Calculate the embedding of each response and the cosine similarity with the explanation
        print("2b/ Calculating embeddings and similarities for responses")
        columns_to_evaluate = [col for col in df.columns if 'response' in col]
        for col in tqdm(columns_to_evaluate, desc="Processing columns"):
            df[f'{col}_truncated'] = df[col].apply(lambda x: truncate_text(x, max_tokens))
            df[f'{col}_tokens'] = df[col].apply(num_tokens_from_string)
            df[f'{col}_embedding'] = df[f'{col}_truncated'].apply(get_embedding).apply(json.dumps)
            df[f'{col}_embedding_sim'] = df.apply(lambda x: cosine_similarity(
                json.loads(x['explanation_embedding']), 
                json.loads(x[f'{col}_embedding'])
            ), axis=1)
        
        # 2c/ Save the DataFrame
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        output_path = os.path.join(os.path.dirname(file_path), f"{base_name}_embed.csv")
        df.to_csv(output_path, index=False)
        print(f"Saved processed data to: {output_path}")

    print("3/ Done processing all DataFrames")

# Usage
process_llm_csvs_GxE(get_embedding, cosine_similarity, truncate_text, num_tokens_from_string)


1/ Loading and preprocessing the data
Loaded and preprocessed: llm_gpt4o.csv
Loaded and preprocessed: llm_gpt4omini.csv
Loaded and preprocessed: llm_gpt3.csv
Loaded and preprocessed: llm_gpt4turbo.csv
2/ Processing DataFrames


Processing files:   0%|          | 0/4 [00:00<?, ?it/s]


Processing: llm_gpt4o.csv
2a/ Adding explanation embeddings
2b/ Calculating embeddings and similarities for responses


Processing files:  25%|██▌       | 1/4 [07:54<23:42, 474.10s/it]

Saved processed data to: /Users/kenzabenkirane/Desktop/GitHub/24ucl_thesis/bias_llm_clinical_challenge/results/experiment1/genderxethnicity/cleaned_results/llm_gpt4o_embed.csv

Processing: llm_gpt4omini.csv
2a/ Adding explanation embeddings
2b/ Calculating embeddings and similarities for responses


...

Saved processed data to: /Users/kenzabenkirane/Desktop/GitHub/24ucl_thesis/bias_llm_clinical_challenge/results/experiment1/genderxethnicity/cleaned_results/llm_gpt4omini_embed.csv

Processing: llm_gpt3.csv
2a/ Adding explanation embeddings
2b/ Calculating embeddings and similarities for responses


Processing files:  75%|███████▌  | 3/4 [21:55<07:04, 424.05s/it]

Saved processed data to: /Users/kenzabenkirane/Desktop/GitHub/24ucl_thesis/bias_llm_clinical_challenge/results/experiment1/genderxethnicity/cleaned_results/llm_gpt3_embed.csv

Processing: llm_gpt4turbo.csv
2a/ Adding explanation embeddings
2b/ Calculating embeddings and similarities for responses


Processing files: 100%|██████████| 4/4 [28:19<00:00, 424.77s/it]

Saved processed data to: /Users/kenzabenkirane/Desktop/GitHub/24ucl_thesis/bias_llm_clinical_challenge/results/experiment1/genderxethnicity/cleaned_results/llm_gpt4turbo_embed.csv
3/ Done processing all DataFrames
